In [22]:
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import torch.optim as optim
from torch.optim import lr_scheduler
from torchmetrics import MeanAbsolutePercentageError

import seaborn as sns
sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

from utility_funcs import *

import dill

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, ConstantKernel, RBF
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import Normalizer

In [23]:
model_type = "network"

In [24]:
class SingleNet(nn.Module):
    '''

    Класс одиночной нейронной сети

    '''
    def __init__(self, output_size, activation=nn.ReLU(), flattened_size=CFG.K * CFG.K):
        '''
        
        FC_type: тип полносвязных слоев: 'regular' / 'simple

        convolution: сверточная часть сети

        '''
        super().__init__()

        self.FC = nn.Sequential(
        #     # nn.BatchNorm1d(flattened_size),

            nn.Linear(flattened_size, 128),
            activation,
            # nn.Dropout(0.3),
            nn.BatchNorm1d(128),

            nn.Linear(128, 256),
            activation,
            # nn.Dropout(0.3),
            nn.BatchNorm1d(256),

            nn.Linear(256, 256),
            activation,
            # nn.Dropout(0.3),
            nn.BatchNorm1d(256),

            nn.Linear(256, 256),
            activation,
            # nn.Dropout(0.3),
            nn.BatchNorm1d(256),

            # nn.Linear(256, 256),
            # activation,
            # # nn.Dropout(0.3),
            # nn.BatchNorm1d(256),
            
            nn.Linear(256, 512),
            activation,
            # nn.Dropout(0.3),
            # nn.BatchNorm1d(512),
            nn.Linear(512, output_size),
        )

        # self.FC = nn.Sequential(
        #     nn.Linear(flattened_size, 64),
        #     activation,

        #     nn.Linear(64, output_size)
        # )

        # self.FC = nn.Linear(flattened_size, output_size)

    def forward(self, x):
        # x - is batch of matrices KxK

        # Здесь происходят какие-то там свертки, пуллинги и тп..

        x = self.FC(x)

        return x

In [25]:
class SklearnModel:
    def __init__(self, model):
        '''
        model - sklearn model
        '''
        self.model = model

    def __call__(self, X):
        return self.model.predict(X)

In [26]:
model_file_path = f'./trained_models/network_{CFG.N}_movements_K{CFG.K}.pickle'
model_vel_file_path = f'./trained_models/network_{CFG.N}_velocities_K{CFG.K}.pickle'

descaler_path = f'./trained_models/descaler_{CFG.N}_K{CFG.K}.pickle'
descaler_vel_path = f'./trained_models/descaler_vel_{CFG.N}_K{CFG.K}.pickle'

In [27]:
with open(model_file_path, 'rb') as handle:
    model = dill.load(handle)

with open(model_vel_file_path, 'rb') as handle:
    model_vel = dill.load(handle)

if model_type == "gpr":
    descaler = Descaler(1, 0)
    descaler_vel = Descaler(1, 0)
    model = SklearnModel(model)
else:
    with open(descaler_path, 'rb') as handle:
        descaler = dill.load(handle)
    

In [35]:
# Пример того как state выглядит:
{}

{}

In [29]:
def make_prediction(X, pinv_A, model, model_type="network", descaler=Descaler(1, 0)):
    return descaler(model(X).detach() @ pinv_A)

In [30]:
def csv_row_to_state(path):
    pass

In [31]:
def fill_csv():
    '''
    fills csv after current step
    '''
    pass

def fill_xyz():
    '''
    fills xyz file after current step
    '''
    pass

In [32]:
def state_into_matrices():
    '''
    returns (X, pinv_A),
    where X is flattened matrix with velocities added at the end
    '''
    pass


In [33]:

def predict_vel(X, v0, model, model_type="network"):
    pass

def predict_movement(X, v, model, model_type="network"):
    pass

def make_step():
    pass

In [34]:
def integration_cycle(starting_coords, starting_vels):
    pass